# Partioning the evolution of uncertainties

In [ ]:
import os
import geopandas as gpd
import xarray as xr 
import numpy as np
import pandas as pd
from oggm import utils
from glob import glob

# variables to analize
variables        = ['melt_off_glacier', 'melt_on_glacier', 'liq_prcp_off_glacier', 'liq_prcp_on_glacier', 'volume', 'area']
variables_final  = ['total_runoff','melt_on_glacier', 'smb']

mods = glob("/home/rooda/OGGM_results/new/*/", recursive = True)
os.chdir("/home/rooda/Dropbox/Patagonia/MS2 Results/")

# historical 
outlines = ["RGI6", "RGI7"]
volume   = ["M22", "F19"]
climate  = ["PMET", "CR2MET", "ERA5","MSWEP"]

#future
gcms = ["ACCESS-CM2", "BCC-CSM2-MR", "CMCC-ESM2", "FGOALS-f3-L", "GFDL-ESM4", "CMCC-CM2-SR5", "KACE-1-0-G", "MPI-ESM1-2-HR", "MRI-ESM2-0", "MIROC6"]
ssps = ["ssp126", "ssp245", "ssp370", "ssp585"]
bcms = ["MVA", "DQM", "MBC"]

## Ids for each glacier

In [ ]:
RGI6_ids = gpd.read_file("/home/rooda/Dropbox/Patagonia/GIS South/Glaciers/RGI6_v2.shp")
RGI7_ids = gpd.read_file("/home/rooda/Dropbox/Patagonia/GIS South/Glaciers/RGI7_v2.shp")
RGI6_ids = RGI6_ids[RGI6_ids.area_km2 > 7][["RGIId", "Zone"]]
RGI7_ids = RGI7_ids[RGI7_ids.area_km2 > 7]

# RGI6 doesnt have IDs yet 
RGI7_ids = utils.cook_rgidf(RGI7_ids, o1_region='17', o2_region='02', bgndate= RGI7_ids.src_date, 
                            version = "70", assign_column_values= {'Zone' : 'Zone'})

RGI7_ids = RGI7_ids[["RGIId", "Zone"]]
ids = pd.concat([RGI6_ids, RGI7_ids]).set_index("RGIId")

dict_zone = {1:'PPY', 2:'PCA', 3:'NPI-E', 4:'NPI-W', 5:'SPI-N', 6:'SPI-C', 7:'SPI-S', 8:'GCN', 9:'CDI'}
ids = ids.replace({"Zone": dict_zone})

In [ ]:
def preprocess(ds): # remove unnecessary variables and coordinates
    return ds.drop_vars(['hydro_year', 'hydro_month', 'calendar_year', 'calendar_month'])[variables]

def hydro_variables(ds): # calculate total_runoff, melt_on_glacier and smb
    ds["total_runoff"] = (ds.melt_off_glacier + ds.melt_on_glacier + ds.liq_prcp_off_glacier + ds.liq_prcp_on_glacier)*1e-9
    ds["melt_on_glacier"] = (ds.melt_on_glacier)*1e-9
    ds["smb"] = (ds.volume.diff(dim = "time") * 900 / ds.area)
    return ds[variables_final]

# Data

## Ensemble

In [ ]:
all_combs = glob("/home/rooda/OGGM_results/new/*/run_output_*.nc", recursive = True)
all_opts   = xr.open_mfdataset(all_combs, combine='nested', concat_dim="options", chunks="auto", parallel=True, preprocess=preprocess)

# assing zone to each glacier and aggregate the result 
ids_subset = ids[ids.index.isin(all_opts.rgi_id.to_pandas().tolist())]
all_opts   = all_opts.assign_coords(rgi_id = ids_subset.Zone.tolist())
all_opts   = all_opts.groupby('rgi_id').sum()

all_opts   = all_opts.chunk("auto")
all_opts   = hydro_variables(all_opts)

all_opts = all_opts.isel(time = slice(0, -1))
all_opts = all_opts.std(dim="options")
all_opts = all_opts.to_dataframe()
all_opts = all_opts.to_csv("variance_ensemble.csv")

## Scenarios (SSPs) and  natural variability (2020-2030)

In [ ]:
period = slice("2020-01-01", "2029-31-12")
all_ssps = []

for mod in mods: 
    for gcm in gcms: 
        for bcm in bcms:
            files  = glob(mod + "*" + gcm + "_*" +  bcm + "*.nc")
            ssp_i  = xr.open_mfdataset(files, combine='nested', concat_dim="options", parallel=True, preprocess=preprocess)

            # assing zone to each glacier and aggregate the result 
            ids_subset = ids[ids.index.isin(ssp_i.rgi_id.to_pandas().tolist())]
            ssp_i   = ssp_i.assign_coords(rgi_id = ids_subset.Zone.tolist())
            ssp_i   = ssp_i.groupby('rgi_id').sum()
 
            ssp_i   = hydro_variables(ssp_i)
            ssp_i = ssp_i.std(dim="options")
            all_ssps.append(ssp_i)
            
all_ssps = xr.concat(all_ssps, dim='ssp')
all_ssps = all_ssps.isel(time = slice(0, -1))

# natural variability
all_ssps_nv = all_ssps.sel(time = period)
all_ssps_nv = all_ssps_nv.mean(dim="ssp")
all_ssps_nv = all_ssps_nv.mean(dim="time")
all_ssps_nv = all_ssps_nv.to_dataframe().to_csv("variance_ssp_nv.csv")

# scenarios
all_ssps = all_ssps.mean(dim="ssp")
all_ssps = all_ssps.to_dataframe().to_csv("variance_ssp_scenarios.csv")

## Models (GCMs)

In [ ]:
all_gcms = []

for mod in mods: 
    for ssp in ssps: 
        for bcm in bcms:
                files = glob(mod + "*" + ssp + "_" + bcm + "*.nc")
                gcm_i = xr.open_mfdataset(files, combine='nested', concat_dim="options", parallel=True, preprocess=preprocess)

                # assing zone to each glacier and aggregate the result 
                ids_subset = ids[ids.index.isin(gcm_i.rgi_id.to_pandas().tolist())]
                gcm_i   = gcm_i.assign_coords(rgi_id = ids_subset.Zone.tolist())
                gcm_i   = gcm_i.groupby('rgi_id').sum()
                
                gcm_i   = hydro_variables(gcm_i)
                gcm_i = gcm_i.std(dim="options")
                all_gcms.append(gcm_i)
    
all_gcms = xr.concat(all_gcms, dim='gcm')
all_gcms = all_gcms.isel(time = slice(0, -1))
all_gcms = all_gcms.mean(dim="gcm")
all_gcms = all_gcms.to_dataframe().to_csv("variance_gcms.csv")

## Model set-up (only climate)

In [ ]:
all_climate = []

for rgi in outlines: 
    for vol in volume: 
        for gcm in gcms: 
            for ssp in ssps: 
                for bcm in bcms:
                    path_i = glob(os.path.dirname(os.path.normpath(mods[0])) + "/*"+ rgi +"*_"+ vol +"*/*" + gcm + "_" + ssp + "_" + bcm + ".nc")
                    mod_i  = xr.open_mfdataset(path_i, combine='nested', concat_dim="options", parallel=True, preprocess=preprocess)

                    # assing zone to each glacier and aggregate the result 
                    ids_subset = ids[ids.index.isin(mod_i.rgi_id.to_pandas().tolist())]   
                    mod_i   = mod_i.assign_coords(rgi_id = ids_subset.Zone.tolist())
                    mod_i   = mod_i.groupby('rgi_id').sum()

                    mod_i   = hydro_variables(mod_i)
                    mod_i = mod_i.std(dim="options")
                    all_climate.append(mod_i)

all_climate = xr.concat(all_climate, dim='climate')
all_climate = all_climate.isel(time = slice(0, -1))
all_climate = all_climate.mean(dim="climate")
all_climate = all_climate.to_dataframe().to_csv("variance_climate.csv")

## Model set-up (geometry/volume)

In [ ]:
all_mods = []

for cli in climate:
    for gcm in gcms: 
        for ssp in ssps: 
            for bcm in bcms:
                path_i = glob(os.path.dirname(os.path.normpath(mods[0])) + "/*" + cli + "*/*" + gcm + "_" + ssp + "_" + bcm + ".nc")
                mod_i  = xr.open_mfdataset(path_i, combine='nested', concat_dim="options")[variables]
                mod_i  = mod_i.drop_vars(['hydro_year', 'hydro_month', 'calendar_year', 'calendar_month'])

                # assing zone to each glacier and aggregate the result 
                ids_subset = ids[ids.index.isin(mod_i.rgi_id.to_pandas().tolist())]   
                mod_i = mod_i.assign_coords(rgi_id = ids_subset.Zone.tolist())
                mod_i = mod_i.groupby('rgi_id').sum()

                mod_i = hydro_variables(mod_i)
                mod_i = mod_i.std(dim="options")
                all_mods.append(mod_i)

all_mods = xr.concat(all_mods, dim='mod')
all_mods = all_mods.isel(time = slice(0, -1))
all_mods = all_mods.mean(dim="mod")
all_mods = all_mods.to_dataframe().to_csv("variance_geometry.csv")